# Demo: Serenity Reference Data API

Serenity has its own asset master containing the system-specific and asset metadata that you need to use the other features.
This notebook demonstrates how to use it to query the system for reference data.

In [ ]:
%%capture --no-stderr --no-display
%load_ext autoreload
%autoreload 2

In [ ]:
from os import getenv
from serenity_sdk.widgets import ConnectWidget

# if you want to auto-connect, set this environment variable to your desired default
connect_widget = ConnectWidget(getenv('SERENITY_CONFIG_ID', None))

In [ ]:
import datetime

import pandas as pd

# create an alias to the api
api = connect_widget.get_api()

## Core Reference Data

Serenity supports multiple asset types to encompass instruments like tokens, fiat currency and derivatives. You can get a simple mapping of the type names and their descriptions via the API:

In [ ]:
asset_types = api.refdata().get_asset_types()
pd.DataFrame.from_dict(asset_types, orient='index', columns=['Description']).reset_index().rename(columns={'index': 'Short ID'})

A symbol authority is an organization that defines a symbology, a universe of symbols for assets. You can use different authorities like DAR or KAIKO for symbols, and then map them into symbols. This API call lets you get their names and descriptions:

In [ ]:
authorities = api.refdata().get_symbol_authorities()
pd.DataFrame.from_dict(authorities, orient='index', columns=['Description']).reset_index().rename(columns={'index': 'Short ID'})

Serenity's risk attribution and other tools can use sector taxonomies to create hierarchies of assets. Because this is extensible, they each have unique identifiers. This call lets you get you those identifiers by name, e.g. DATS:

In [ ]:
sector_taxonomies = api.refdata().get_sector_taxonomies()
pd.DataFrame.from_dict(sector_taxonomies, orient='index', columns=['Short ID']).reset_index().rename(columns={'index': 'UUID'})

If you just want to browse the asset catalog, you can use this call to get all the asset records (in JSON format):

In [ ]:
asset_summaries = api.refdata().get_asset_summaries()
asset_summaries = [{key: value for key, value in summary.items() if key != 'xrefSymbols'} for summary in asset_summaries]
pd.json_normalize(asset_summaries)

Finally, the most common use for reference data is to map from your own system's identifiers to Serenity's identifiers. There is a helper class that does this for you called AssetMaster. Here are three different mappings using different symbologies:

In [ ]:
asset_master = api.refdata().load_asset_master()

portfolio_raw = {
    'BTC': 100,
    'ETH': 1000
}
portfolio = asset_master.create_portfolio(portfolio_raw, symbology='NATIVE')
positions = portfolio.to_asset_positions()
pd.json_normalize(positions)

In [ ]:
portfolio_raw = {
    'tok.btc.bitcoin': 100,
    'tok.eth.ethereum': 1000
}
portfolio = asset_master.create_portfolio(portfolio_raw, symbology='SERENITY')
positions = portfolio.to_asset_positions()
pd.json_normalize(positions)

In [ ]:
portfolio_raw = {
    'bitcoin': 100,
    'ethereum': 1000
}
portfolio = asset_master.create_portfolio(portfolio_raw, symbology='COINGECKO')
positions = portfolio.to_asset_positions()
pd.json_normalize(positions)

## Derivatives Reference Data

In 2023 we will be redoing our reference data API to more efficiently handle the tens of thousands of derivatives contracts in our database, but as a tactical measure the new Pricer API for options
offers supporting methods to use in combination with that API's inputs and outputs. At this time all it offers in BTC and ETH options and futures on Deribit. NOTE: at this time though we have expired
options, the as_of_date parameter only supports latest date; this will also be addressed in early 2023.

We support USD as a base currency:

In [ ]:
api.pricer().get_supported_currencies()

Bitcoin and Ethereum as underliers:

In [ ]:
underliers = api.pricer().get_supported_underliers()
print(underliers)

All live futures and options on those underliers on Deribit:

In [ ]:
for underlier in underliers:
    display(f"Currently {len(api.pricer().get_supported_futures(underlier_asset_id=underlier.asset_id))} Deribit futures on {underlier.native_symbol}")
    display(f"Currently {len(api.pricer().get_supported_options(underlier_asset_id=underlier.asset_id))} Deribit options on {underlier.native_symbol}")